In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
import re
import seaborn as sns
from datetime import date, timedelta
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import regularizers
import tensorflow as tf
import tensorflow.keras.backend as K

In [12]:
info_compet = pd.read_csv(os.path.join('new_competition_info.csv'), encoding = 'euc-kr')
info_login = pd.read_csv(os.path.join('new_login_info.csv'), encoding = 'euc-kr')
info_user = pd.read_csv(os.path.join('new_user_info.csv'), encoding='euc-kr')
info_sub = pd.read_csv(os.path.join('new_submission_info.csv'), encoding= 'euc-kr')


C:\Users\82109\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
def pipline(info_sub, info_compet, info_user, train):
    
    #info sub -----------------------------------------------------------------

    info_sub['c_time'] = pd.to_datetime(info_sub['c_time'])
    info_sub['date'] = info_sub.c_time.dt.date
    info_sub['Weekday'] = info_sub['c_time'].dt.weekday
    info_sub['YearMonth'] = info_sub['c_time'].dt.to_period('M')

    day_group = info_sub.groupby(['YearMonth','date'])

    id_col = ['sub_id']

    info_group_datas = pd.concat([
        day_group[id_col].count().rename(columns = lambda s : 'counts_' + s)
    ], axis = 1).reset_index()

    info_group_datas.drop('YearMonth', axis = 1, inplace = True)


    # info login -----------------------------------------------------------------

    info_login.dropna(inplace = True)

    info_login['c_time'] = pd.to_datetime(info_login['c_time'])
    info_login['date'] = info_login.c_time.dt.date
    info_login['YearMonth'] = info_login.c_time.dt.to_period('M')


    login_group = info_login.groupby(['YearMonth','date'])
    login_col = ['login_id']

    login_group_data = pd.concat([
        login_group[login_col].count().rename(columns = lambda s : 'loginsize.' + s)], axis = 1).reset_index()

    login_group_data.drop('YearMonth', axis = 1, inplace = True)

    from datetime import date, timedelta
    info_compet['period_start'] = pd.to_datetime(info_compet['period_start'])
    info_compet['merge_deadline'] = pd.to_datetime(info_compet['merge_deadline'])

    dayframe = pd.DataFrame(columns = {'period_start'})
    
    # info compet -----------------------------------------------------------------
    days = []
    def daterange(d1, d2):
        for i in range((d2 - d1).days + 1):
            d3 = d1 + timedelta(days=i)
            days.append(d3)

    for i in range(0, len(info_compet)):
        daterange(info_compet['period_start'][i], info_compet['merge_deadline'][i])

    dayframe['period_start'] = days
    result = pd.merge(dayframe, info_compet, how = 'outer')

    info_compet.fillna(0, inplace = True)

    info_compet['period_start'] = pd.to_datetime(info_compet['period_start'])
    info_compet['date'] = info_compet.period_start.dt.date

    traind = info_compet.groupby(['date'])
    train_col = ['prize','max_file_per_day']
    train_group_compet = pd.concat([
        traind[train_col].sum().rename(columns = lambda s : 'dmean_' + s)
    ], axis = 1).reset_index()



    # info user -----------------------------------------------------------------

    info_user.dropna(inplace = True)

    info_user['c_time'] = pd.to_datetime(info_user['c_time'])
    info_user['date'] = info_user.c_time.dt.date
    info_user['YearMonth'] = info_user.c_time.dt.to_period('M')

    year_month = info_user.groupby(['YearMonth','date'])

    col_a = ['id']
    a_col = ['entered_competition_cnt', 'talk_board_cnt']

    info_user_group = pd.concat([
        year_month[col_a].count().rename(columns = lambda s : 'dcount_' + s),
        year_month[a_col].sum().rename(columns = lambda s : 'dmean_' + s)
    ],axis = 1).reset_index()

    info_user_group.drop('YearMonth', axis = 1, inplace = True)


    # train -----------------------------------------------------------------

    train['DateTime'] = pd.to_datetime(train['DateTime'])
    train['date'] = train.DateTime.dt.date
    train_final = train.groupby('date').sum().reset_index()

    train['Year'] = train['DateTime'].dt.year
    train['WeekDay'] = train['DateTime'].dt.weekday
    train['Day'] = train['DateTime'].dt.day
    train['YearMonth'] = train['DateTime'].dt.to_period('M')

    traind = train.groupby(['YearMonth','date'])

    train_col = ['Users', 'Sessions', 'NewVisitors', 'PageViews']

    train_groups = pd.concat([
        traind[train_col].mean().rename(columns = lambda s : 'dmean_' + s),
        traind[train_col].sum().rename(columns = lambda s : 'dsum_' + s)
    ], axis = 1).reset_index()


    #train_md = pd.merge(train_groups, train_groupm, on ='YearMonth', how = 'left')
    train_groups.drop('YearMonth', axis = 1, inplace = True)
    train_groups

    train = pd.merge(train_final, train_groups, on = 'date', how = 'left')
    train_sub = pd.merge(train, info_group_datas, on = ['date'], how = 'left')
    train_login = pd.merge(train_sub,login_group_data, on = ['date'], how ='left')
    train_user = pd.merge(train_login, info_user_group, on = ['date'], how = 'left')
    train = pd.merge(train_user, train_group_compet, on = ['date'], how = 'left')
    train.fillna(0, inplace = True)
    
    return train

train = pipline(info_sub, info_compet, info_user, train)



In [ ]:
mini = train.iloc[:,1:].min()
size = train.iloc[:,1:].max() - train.iloc[:,1:].min()
train.iloc[:,1:] = (train.iloc[:,1:] -  mini) / size

input_window = 30
output_window = 7

window_x = np.zeros((train.shape[0] - (input_window + output_window), input_window, 19))
window_y = np.zeros((train.shape[0] - (input_window + output_window), output_window, 19))

for start in range(train.shape[0] - (input_window + output_window)):
    end = start + input_window    
    window_x[start,:, :] = train.iloc[start : end                , 1: ].values
    window_y[start,:, :] = train.iloc[end   : end + output_window, 1: ].values
window_x.shape


In [ ]:
# (size, timestep, feature) 
inputs = Input(shape=(None, 15))
# print(inputs.shape)
x = Conv1D(30, kernel_size=5, padding='causal')(inputs)
x = LSTM(30, return_sequences=True)(x)
x = LSTM(30, return_sequences=True)(x)
# x = LSTM(30, return_sequences=True)(x)
out_time = Dense(4)(x[:,-7:, :]) # Dense(추출하는 컬럼 수) x[:, -출력 윈도사이즈, :]
out_time = K.reshape(out_time, (-1,7,4)) # -1, 아웃_윈도사이즈, 추출 컬럼 수
model = Model(inputs=inputs, outputs=out_time)



In [ ]:
lr_minus = ReduceLROnPlateau(monitor='val_loss', patience=40, verbose=1, min_lr=1e-7, factor=0.1) # lr_m 40  / es 80 / lr 2e-2 / 400 epochs
ckpt_path = './LSTM.ckpt'
check = ModelCheckpoint(ckpt_path,
                       save_best_only=True,
                       save_weights_only=True,
                       monitor='val_loss',
                       mode='min',
                       verbose=1)
es = EarlyStopping(patience=80, monitor='val_loss') # 60 -> 80

In [ ]:
model.compile(optimizer=Adam(1e-2) ,loss= 'mse') # ADAM 2e-2 : 0.017 => 3.34

In [ ]:
history = model.fit(x=window_x, y=window_y, epochs=400, verbose=0, callbacks=[check, es, lr_minus], validation_split=0.2)

In [ ]:
fig, ax = plt.subplots()
ax.plot(history.history['loss'], 'b', label = 'loss')
ax.plot(history.history['val_loss'], 'r', label='val_loss')
ax.set_xlabel('epoch')
ax.set_ylabel('loss')
ax.legend(loc='upper left')

plt.show()

In [ ]:
# last_month = tf.convert_to_tensor(window_x[-1,:,:][np.newaxis,...])




last_month = train.iloc[-30:,:].values[np.newaxis,...]
last_month = tf.convert_to_tensor(last_month)


for start in range((len(submission) - output_window)//7 + 2):
    start = start * 7
    next_week = model.predict(last_month)
    last_month = tf.concat([last_month[:,7:,:], next_week], axis = 1)
    
    pred_week = next_week.reshape(output_window,4)
    pred_week = pred_week * size.values + mini.values
    pred_week = pred_week.astype(int)
    
    if start/7 == (len(submission) - output_window)//7 + 1:
        submission.iloc[start :, 1:] = pred_week[-submission.iloc[start :, 1:].shape[0]:,:]
    else:
        submission.iloc[start : start + output_window, 1:] = pred_week
submission